In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import os
import re
from fuzzywuzzy import process
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'fuzzywuzzy'

In [ ]:
%load_ext autoreload
%autoreload 2
from funciones.funciones import check_df
from funciones.funciones import identificacion_valores_problem

Análisis de los datos de campañas de marketing de una empresa Fintech para
conseguir identificar patrones, tendencias y factores que influyen en que un
cliente acabe contratando un depósito. Tras un análisis exhaustivo de todos sus
datos, habrá que implementar un modelo simple que ayude a entender y predecir
la efectividad de las campañas de marketing, y diseñar un dashboard para la
capa ejecutiva de la fintech con los principales insights y KPIs de los resultados
de las campañas.

In [ ]:
os.getcwd()

In [ ]:
df = pd.read_excel('bank-additional_bank-additional-full.xlsx')

# **1. Análisis de datos: Índice (PDF)**


- Entendimiento de datos de clientes, de los detalles de las
campañas y sus resultados
- Estadísticas descriptivas para entender las características básicas
de los datos.
- Análisis de la relación entre características demográficas del
cliente (edad, trabajo, educación) y la suscripción a un depósito a
plazo.
- Evaluación del impacto de los detalles de la campaña (número de
contactos, mes, día de la semana) en el resultado de la campaña.

Cada punto de ahora en adelante es un punto del guión del trabajo

## **1.1. Entendimiento de datos de clientes**

### **Datos del cliente**

* **age:** Edad

* **job:** Profesión (admin, blue-collar, technician, etc.)

* **marital:** Estado civil (married, single, divorced)

* **education:** Nivel de estudios

* **default:** ¿Ha tenido impagos de crédito? (yes/no/unknown)

* **housing:** ¿Tiene hipoteca? (yes/no/unknown)

* **loan:** ¿Tiene préstamo personal? (yes/no/unknown)


### **Datos de la campaña actual**

* **contact:** Tipo de contacto (cellular/telephone)

* **month:** Mes del último contacto

* **day_of_week:** Día de la semana del último contacto

* **duration:** Duración de la llamada en segundos

* **campaign:** Número de contactos realizados en esta campaña para este cliente


### **Datos de campañas anteriores**

* **pdays:** Días desde el último contacto de una campaña anterior (999 = nunca contactado antes)

* **previous:** Número de contactos en campañas anteriores

* **poutcome:** Resultado de la campaña anterior (success/failure/nonexistent)


### **Datos de indicadores macroeconómicos**

* **emp.var.rate (Employment Variation Rate):**  Tasa de variación del empleo. Mide el cambio porcentual en el nivel de empleo respecto al trimestre anterior. Un valor negativo indica destrucción de empleo, positivo indica creación.

* **cons.price.idx (Consumer Price Index):**   Índice de precios al consumo. Básicamente, la inflación. Valores más altos = cosas más caras.

* **cons.conf.idx(Consumer Confidence Index):** Índice de confianza del consumidor. Mide el optimismo/pesimismo de los hogares sobre la economía. Suele oscilar entre -50 y +50 aproximadamente, donde negativo = pesimismo.

* **euribor3m:**  El Euríbor a 3 meses. El tipo de interés al que los bancos europeos se prestan dinero entre sí. Afecta directamente a hipotecas y productos financieros.

* **nr.employed:** Número de empleados (en miles, normalmente). El volumen total de personas empleadas en el país en ese momento.


### **Contratación o no**

* **y**: Sí ha contratado o no el depósito.

___________________________

## 1.2 **Estadísticas descriptivas**

In [ ]:
check_df(df)

**¿Qué valores tienen las columnas categóricas?**

In [ ]:
for campo_categorico in df:
    if df[campo_categorico].dtype == 'object':
        print(campo_categorico)
        print(df[campo_categorico].unique())
        print('______')

**¿Qué valores tienen las columnas numéricas?**

In [ ]:
for campo_numerico in df:
    if df[campo_numerico].dtype in ['int64', 'float64']:
        print(campo_numerico)
        print(df[campo_numerico].unique())
        print('______')

________________________________

## **1.3. Limpieza de datos**

Vemos que hay bastantes campos que renombrar, tipos de datos que hay que cambiar y datos erróneos. Analizamos primero qué tendríamos que revisar a primera vista.

### **1.3.1. Campos a limpiar**


- **Ordenar**
    - Month
    - Day
    - Education
- **Reformatear**
    - Valores
        - Education
            - ¿Cambiamos los valores?
        - Día
            - Nombre completo y en primera mayuscula
        - Job
        - emp.var.rate
            - Hay mezcla de fechas, ints y floats
            - Debería ser float

      ~~- nr.employed~~
            ~~- Está en formato fecha~~
            ~~-Tendría que ser int~~
            


**Preguntas al profe**

- El 95% de los clientes no tuvieron contacto previo, y tienen el valor 999. ¿Qué hacemos con él?

_____________________________________________________________________________________________________________

### **1.3.2. Valores problemáticos**

#### **1.3.2.1. Nulos y Outliers**

In [ ]:
identificacion_valores_problem(df)

#### **1.3.2.2. Duplicados**

¿Cuántos duplicados hay?

In [ ]:
df.duplicated().sum()

¿Dónde están los duplicados?

In [ ]:
df[df.duplicated(keep=False)]

### **1.3.3. Limpieza**

#### **1.3.3.1. Renombrar campos**

Renombramos columnas y llamamos al df "df_clean"

In [ ]:
df_clean = df.copy()
df_clean = df_clean.rename(columns={'y':'result',
                                    'pdays':'days_last_contact',
                                    'emp.var.rate':'employment_variation_rate',
                                    'previous':'number_previous_contacts',
                                    'nr.employed':'nr_employed',
                                    'housing':'mortgage',
                                    'default':'credit_default',
                                    'campaign':'number_of_contacts',
                                    'cons.conf.idx':'cons_conf_idx',
                                    'cons.price.idx':'cons_price_idx'})
df_clean.columns

#### **1.3.3.2. Limpieza de duplicados**

Solo hay 24 registros duplicados, no afecta a la muestra total. Yo los dropearía.

In [ ]:
df_clean = df_clean.drop_duplicates(keep='first')

In [ ]:
df_clean.duplicated().sum()

#### **1.3.3.3. Limpieza de nulos**

In [ ]:
df_clean.isna().sum()

In [ ]:
df_clean[['credit_default', 'mortgage', 'loan']].apply(pd.Series.value_counts)

# Tratando unknown como nulos

Los 'nulos' de hipoteca y prestamo no me parecen relevantes pero los de credit_default me parecen muchos, quito solo los pequeños

In [ ]:
df_clean = df_clean[(df_clean['mortgage'] != 'unknown') & (df_clean['loan'] != 'unknown')]

#### **1.3.3.4. Cambiar tipos de datos**

**nr_employed**

In [ ]:
mask = df_clean['nr_employed'].apply(lambda x: 'datetime' in str(type(x)))
# Obtiene una lista de True/False sobre si los registros contienen "datetime" en su type.


df_clean.loc[mask, 'nr_employed'] = df_clean.loc[mask, 'nr_employed'].apply(lambda x: x.year)
# A los registros de la columna nr_employed que cumplen las condiciones de la variable anterior, se les aplicará la extracción del año, y se introducirá en esos mismos registros.

**Euribor**

Limpio la columna de euribor, todo tipo float representado en estilo x.xxx, como debe ser

In [3]:
from funciones.funciones import eur_good
df_clean['euribor3m'] = df_clean['euribor3m'].apply(eur_good)

ModuleNotFoundError: No module named 'funciones'

In [4]:
df_clean['euribor3m'] = df_clean['euribor3m'].map(lambda x: f"{x:.3f}")

NameError: name 'df_clean' is not defined

In [5]:
df_clean['euribor3m'].astype(float).mean()

NameError: name 'df_clean' is not defined

#### **1.3.3.5. Limpieza de outliers**

Los mantenemos, pero creamos una tabla nueva para cuando haya que visualizar datos no haya datos disonantes.

In [ ]:
identificacion_valores_problem(df_clean)

Creamos la tabla dentro del rango intercuartilico

In [ ]:
df_clean_no_outliers = df_clean.copy()

for col in df_clean_no_outliers.select_dtypes(include='number').columns:
    Q1 = df_clean_no_outliers[col].quantile(0.25)
    Q3 = df_clean_no_outliers[col].quantile(0.75)
    IQR = Q3 - Q1
    df_clean_no_outliers = df_clean_no_outliers[(df_clean_no_outliers[col] >= Q1 - 1.5 * IQR) & (df_clean_no_outliers[col] <= Q3 + 1.5 * IQR)]

Comprobamos que hemos eliminado los outliers

In [ ]:
identificacion_valores_problem(df_clean_no_outliers) # Ojo que los datos que no estén transformados a numérico no los quita de outliers.

## 1.4. **Análisis de la relación entre características demográficas del cliente (edad, trabajo, educación) y la suscripción a un depósito a plazo.**

## **1.5. Evaluación del impacto de los detalles de la campaña (número de contactos, mes, día de la semana) en el resultado de la campaña.**

**¿Cuál es la proporción de los usuarios que contrataron un depósito?**

In [ ]:
df_yes_no = df['y'].value_counts(normalize=True)*100

df_yes_no

In [ ]:
df['y'].value_counts().plot.pie()

**¿Cuál es la distribución de las llamadas en meses?**

In [ ]:
sns.countplot(data=df, x='month', order=df['month'].value_counts().index)

**¿Cuántas veces se contacta al cliente? Distribución**

In [ ]:
sns.histplot(df['campaign'], bins=6)

**¿Existe correlación entre la contratación del plazo y el euribor?**

In [ ]:
df_euribor_yes = df[['euribor3m','y']]

df_euribor_yes.corr()

### El formato está corrupto, no se puede calcular aún